## **Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from glob import glob
from datetime import datetime
import numpy as np
import random
import cv2
import os

import tensorflow as tf
print('Tensorflow version: {}'.format(tf.__version__))

Tensorflow version: 1.15.0


## **FCN**

In [0]:
class FCNVGG:
    # ---------------------------------------------------------------------------
    def __init__(self, session, num_classes):
        self.session = session
        self.num_classes = num_classes

    def build_from_metagraph(self, metagraph_file, checkpoint_file):
        self.__load_fcnvgg(metagraph_file, checkpoint_file)

    def build_from_vgg(self, vgg_path):
        self.__load_vgg(vgg_path)
        self.__make_result_tensors()

    def __load_fcnvgg(self, metagraph_file, checkpoint_file):
        saver = tf.train.import_meta_graph(metagraph_file)
        saver.restore(self.session, checkpoint_file)

        self.image_input = self.session.graph.get_tensor_by_name('image_input:0')
        self.keep_prob = self.session.graph.get_tensor_by_name('keep_prob:0')
        self.logits      = self.session.graph.get_tensor_by_name('sum/fcn_logits:0')
        self.softmax     = self.session.graph.get_tensor_by_name('result/fcn_softmax:0')
        self.classes     = self.session.graph.get_tensor_by_name('result/fcn_class:0')


    def __load_vgg(self, vgg_path):
        model = tf.saved_model.loader.load(self.session, ['vgg16'], vgg_path)
        graph = tf.get_default_graph()

        self.image_input = graph.get_tensor_by_name('image_input:0')
        self.keep_prob = graph.get_tensor_by_name('keep_prob:0')
        self.vgg_layer3 = graph.get_tensor_by_name('layer3_out:0')
        self.vgg_layer4 = graph.get_tensor_by_name('layer4_out:0')
        self.vgg_layer7 = graph.get_tensor_by_name('layer7_out:0')

    def __make_result_tensors(self):
        """
        :param correct_label:
        :param num_classes:
        :return:
        """
        # Use a shorter variable name for simplicity
        layer3, layer4, layer7 = self.vgg_layer3, self.vgg_layer4, self.vgg_layer7
        # Convert FCN to CONV
        # Apply 1x1 convolution in place of fully connected layer
        fcn8 = tf.layers.conv2d(layer7, filters=self.num_classes, kernel_size=1, name="fcn8")
        # Up-sample fcn8 with size depth=(4096?) to match size of layer 4
        # so that we can add skip connection with 4th layer
        fcn9 = tf.layers.conv2d_transpose(fcn8, filters=layer4.get_shape().as_list()[-1],
                                          kernel_size=4, strides=(2, 2), padding='SAME', name="fcn9")
        # Add a skip connection between current final layer fcn8 and 4th layer
        fcn9_skip_connected = tf.add(fcn9, layer4, name="fcn9_plus_vgg_layer4")
        # print(fcn9_skip_connected.shape)
        # Up-sample again
        fcn10 = tf.layers.conv2d_transpose(fcn9_skip_connected, filters=layer3.get_shape().as_list()[-1],
                                           kernel_size=4, strides=(2, 2), padding='SAME', name="fcn10_conv2d")

        # Add skip connection
        fcn10_skip_connected = tf.add(fcn10, layer3, name="fcn10_plus_vgg_layer3")

        # Up-sample again
        # Final output: fcn11 = 8 * (4 * layer_out7 + 2 * layer_out4 + layer_out3)
        fcn11 = tf.layers.conv2d_transpose(fcn10_skip_connected, filters=self.num_classes,
                                           kernel_size=16, strides=(8, 8), padding='SAME', name="fcn11")

        with tf.variable_scope('sum'):
            self.logits = tf.reshape(fcn11, (-1, self.num_classes), name="fcn_logits")

        with tf.name_scope('result'):
            self.softmax = tf.nn.softmax(self.logits, name="fcn_softmax")
            self.classes = tf.argmax(self.softmax, axis=-1, name="fcn_class")


    def get_optimizer(self, labels, learning_rate=0.0001):
        with tf.variable_scope('reshape'):
            labels_reshaped  = tf.reshape(labels, [-1, self.num_classes])
            logits_reshaped  = tf.reshape(self.logits, [-1, self.num_classes])

            losses          = tf.nn.softmax_cross_entropy_with_logits(
                                  labels=labels_reshaped,
                                  logits=logits_reshaped)
            loss            = tf.reduce_mean(losses)

        with tf.variable_scope('optimizer'):
            optimizer       = tf.train.AdamOptimizer(learning_rate)
            optimizer       = optimizer.minimize(loss)

        return optimizer, loss

## **VOC**

In [0]:
def color_map(N=256, normalized=False):
    def bitget(byteval, idx):
        return (byteval & (1 << idx)) != 0

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7 - j)
            g = g | (bitget(c, 1) << 7 - j)
            b = b | (bitget(c, 2) << 7 - j)
            c = c >> 3

        cmap[i] = np.array([b, g, r])

    cmap = cmap / 255 if normalized else cmap
    return cmap


def color_map_dict():
    labels = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair',
              'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
              'train', 'tvmonitor', 'void']
    
    return dict(zip(labels[:-1], color_map()[:-1]))

In [24]:
print(color_map_dict())

{'background': array([0, 0, 0], dtype=uint8), 'aeroplane': array([  0,   0, 128], dtype=uint8), 'bicycle': array([  0, 128,   0], dtype=uint8), 'bird': array([  0, 128, 128], dtype=uint8), 'boat': array([128,   0,   0], dtype=uint8), 'bottle': array([128,   0, 128], dtype=uint8), 'bus': array([128, 128,   0], dtype=uint8), 'car': array([128, 128, 128], dtype=uint8), 'cat': array([ 0,  0, 64], dtype=uint8), 'chair': array([  0,   0, 192], dtype=uint8), 'cow': array([  0, 128,  64], dtype=uint8), 'diningtable': array([  0, 128, 192], dtype=uint8), 'dog': array([128,   0,  64], dtype=uint8), 'horse': array([128,   0, 192], dtype=uint8), 'motorbike': array([128, 128,  64], dtype=uint8), 'person': array([128, 128, 192], dtype=uint8), 'pottedplant': array([ 0, 64,  0], dtype=uint8), 'sheep': array([  0,  64, 128], dtype=uint8), 'sofa': array([  0, 192,   0], dtype=uint8), 'train': array([  0, 192, 128], dtype=uint8), 'tvmonitor': array([128,  64,   0], dtype=uint8)}


In [0]:
class VOCSource(object):
    def __init__(self):
        self.num_training = None
        self.num_validation = None
        self.train_generator = None
        self.valid_generator = None

        self.num_testing = None
        self.test_generator = None

        self.label_colors = color_map_dict()
        self.num_classes = len(self.label_colors)
        self.image_size = (224, 224)

    # -------------------------------------------------------------------------------
    def load_data(self, data_dir, images_txt, validation_size=None):
        images_dir = os.path.join(data_dir, 'VOC2012/JPEGImages/')   # very large
        labels_dir = os.path.join(data_dir, 'VOC2012/SegmentationClass/')  # 2913
        print(len(glob(images_dir)))
        print(len(glob(labels_dir)))

        # train.txt: 1464 , val.txt: 1449 , Segmentation: 2913
        with open(images_txt, 'r') as f:
            image_names = f.readlines()
            image_paths, label_paths = [], {}
            for image in image_names:
                image_path = images_dir + image[:-1] + '.jpg'
                image_paths.append(image_path)
                label_paths[os.path.basename(image_path)] = labels_dir + image[:-1] + '.png'
        
        random.shuffle(image_paths)
        if validation_size is None:
            test_images = image_paths
            self.num_testing = len(test_images)
            self.test_generator = self.batch_generator(test_images, label_paths)
        else:
            num_images = len(image_paths)
            valid_images = image_paths[:int(validation_size * num_images)]
            train_images = image_paths[int(validation_size * num_images):]

            self.num_training = len(train_images)
            self.num_validation = len(valid_images)
            self.train_generator = self.batch_generator(train_images, label_paths)
            self.valid_generator = self.batch_generator(valid_images, label_paths)

    # -------------------------------------------------------------------------------
    def batch_generator(self, image_paths, label_paths):
        def gen_batch(batch_size):
            """
            :param batch_size:
            :return: generator of a batch contain (images, labels)
            """
            random.shuffle(image_paths)
            for offset in range(0, len(image_paths), batch_size):
                files = image_paths[offset:(offset + batch_size)]
                images = []
                labels = []
                for image_file in files:
                    label_file = label_paths[os.path.basename(image_file)]  # base name get file name from path
                    image = cv2.resize(cv2.imread(image_file), self.image_size)
                    label = cv2.resize(cv2.imread(label_file), self.image_size)

                    label_bg   = np.zeros([image.shape[0], image.shape[1]], dtype=bool)
                    label_list = []
                    for obj, color_bgr in self.label_colors.items():
                        if obj == 'background':
                            continue
                        label_current  = np.all(label == color_bgr, axis=2)
                        label_bg      |= label_current
                        label_list.append(label_current)

                    label_bg   = ~label_bg
                    label_all  = np.dstack([label_bg, *label_list])
                    label_all  = label_all.astype(np.float32)

                    # label = cv2.resize(cv2.imread(label_file), self.image_size)
                    # label_obj = []
                    # for obj, color_bgr in self.label_colors.items():
                    #     obj = np.all(label == color_bgr, axis=2)
                    #     label_obj.append(obj)

                    # label_all = np.dstack(label_obj)
                    # label_all = label_all.astype(np.float32)

                    images.append(image.astype(np.float32))
                    labels.append(label_all)
                    
                image = np.asarray(image, np.float32)
                yield np.array(images), np.array(labels)

        return gen_batch

## **Utils**

In [0]:
# from source_kitti import KittiSource
# from source_voc import VOCSource
# from fcnvgg import FCNVGG


def load_kitti_source():
    return KittiSource()


def load_voc_source():
    return VOCSource()


def load_fcnvgg(session, num_classes):
    return FCNVGG(session, num_classes)


In [0]:
data_dir = '/content/drive/My Drive/dataset/FCN/VOCdevkit/'
vgg_dir = '/content/drive/My Drive/dataset/FCN/vgg/'
output_dir = '/content/drive/My Drive/dataset/FCN/output/'

log_dir = '/content/drive/My Drive/dataset/FCN/graphs/'
model_dir = '/content/drive/My Drive/dataset/FCN/saved_model/20200213-075842/'

test_txt = '/content/drive/My Drive/dataset/FCN/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt'

## **Evalute**

In [7]:
import sys
import matplotlib.pyplot as plt

batch_size = 12
source = load_voc_source()
source.load_data(data_dir, test_txt)
test_generator = source.test_generator

1
1


In [0]:
# img_label.shape, gt_label.shape

In [0]:
def cal_accuracy(eval_img, gt_img):
    return np.array([
      cal_pixel_accuracy(img_classed[idx], gt_batch[idx]),
      cal_mean_pixel_accuracy(img_classed[idx], gt_batch[idx]),
      cal_mean_IU(img_classed[idx], gt_batch[idx])
    ])

# Intersection-Over-Union accuracy
def cal_pixel_accuracy(eval_img, gt_img):
    """
        Calculate accuracy for all class one times:
            - Extract all class appear in ground truth image
            - sum_ii: pixel of class appear in both ground trush && evalute
            - sum_i : pixel of class appear in ground truth

        input: (224, 224) 
        return: pixel_accuracy
    """
    classes, num_classes = extract_classes(gt_img)
    eval_mask = extract_masks(eval_img, classes, num_classes)
    gt_mask   = extract_masks(gt_img, classes, num_classes)
    
    sum_ii = 0
    sum_i = 0
    
    # classes in ground truth image
    for idx, cl in enumerate(classes):
        curr_eval_mask = eval_mask[idx, :, :]
        curr_gt_mask = gt_mask[idx, :, :]

        sum_ii += np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))  
        sum_i  += np.sum(curr_gt_mask)
    
    if sum_i == 0:
        pixel_accuracy = 0
    else:        
        pixel_accuracy = sum_ii / sum_i
    
    return pixel_accuracy

def cal_mean_pixel_accuracy(eval_img, gt_img):
    """
        Calculate accurcay for each class, then return average:
            - Extract all class appear in ground truth image
            - sum_ii: pixel of class appear in both ground trush && evalute
            - sum_i : pixel of class appear in ground truth

        input: (224, 224) 
        return mean_pixel_accuracy = sum_ii / sum_i
    """
    classes, num_classes = extract_classes(gt_img)
    eval_mask = extract_masks(eval_img, classes, num_classes)
    gt_mask   = extract_masks(gt_img, classes, num_classes)

    accuracy = list([0]) * num_classes

    # classes in ground truth image
    for idx, cl in enumerate(classes):
        curr_eval_mask = eval_mask[idx, :, :]
        curr_gt_mask = gt_mask[idx, :, :]

        sum_ii = np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))  
        sum_i  = np.sum(curr_gt_mask)
    
        if sum_i != 0:
            accuracy[idx] = sum_ii / sum_i

    mean_accuracy = np.mean(accuracy)
    return mean_accuracy

def cal_mean_IU(eval_img, gt_img):
    """
        Calculte with union class between eval_img & gt_img
    """
    classes, num_classes = union_classes(eval_img, gt_img)
    _, num_classes_gt = extract_classes(gt_img)

    eval_mask = extract_masks(eval_img, classes, num_classes)
    gt_mask   = extract_masks(gt_img, classes, num_classes)

    IU = list([0]) * num_classes  # union classes

    for idx, cl in enumerate(classes):
        curr_eval_mask = eval_mask[idx, :, :]
        curr_gt_mask = gt_mask[idx, :, :]

        if (np.sum(curr_eval_mask) == 0) or (np.sum(curr_gt_mask) == 0):  # 
            continue
        
        sum_ii = np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))
        sum_i  = np.sum(curr_gt_mask)
        sum_ij = np.sum(curr_eval_mask)

        IU[idx] = sum_ii / (sum_i + sum_ij - sum_ii)

    mean_IU = np.sum(IU) / num_classes_gt
    return mean_IU

In [0]:
def extract_classes(gt_img):
    classes = np.unique(gt_img) # 0, 1, 2, .., 20
    number_classes = len(classes)

    return classes, number_classes

def extract_masks(img, classes, number_classes):
    height, width  = img.shape[0],  img.shape[1]
    masks = np.zeros((number_classes, height, width)) # classes by ground truth image

    for idx, cl in enumerate(classes):
        masks[idx, :, :] = img == cl

    return masks

def union_classes(eval_img, gt_img):
    eval_classes, _ = extract_classes(eval_img)
    gt_classes, _   = extract_classes(gt_img)

    classes = np.union1d(eval_classes, gt_classes)
    num_classes = len(classes)

    return classes, num_classes

In [20]:
from tqdm import tqdm
import math

def draw_labels(img, label, label_colors):
    img_save = np.zeros_like(img)
    for val, color_bgr in label_colors.items(): # diningtable => rgb(124, 124, 0)
        label_mask = label == list(label_colors.keys()).index(val)
        img_save[label_mask] = color_bgr
    return img_save

state = tf.train.get_checkpoint_state(model_dir)
if state is None:
    print('[!] No network state found in ' + model_dir)
    sys.exit(1)

checkpoint_file = state.all_model_checkpoint_paths[-1]
metagraph_file = checkpoint_file + '.meta'
session_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
with tf.Session(config=session_config) as session:
    session.run(tf.global_variables_initializer())

    net = load_fcnvgg(session, source.num_classes)
    net.build_from_metagraph(metagraph_file, checkpoint_file)
    generator = test_generator(batch_size)

    acc = np.array([0, 0 , 0], dtype=np.float64)
    num_img = 0
    
    n_test_batches = int(math.ceil(source.num_testing/batch_size))
    description = '[i] Processing evalute'

    for X_batch, gt_batch in tqdm(generator, total=n_test_batches, desc=description, unit='batches'):
        gt_batch = np.argmax(gt_batch, axis=3)
        
        feed = {net.image_input: X_batch, net.keep_prob: 1}
        img_classed = session.run(net.classes, feed_dict=feed)
        img_classed = img_classed.reshape(-1, source.image_size[1], 
                                          source.image_size[0])
        batch_acc = np.array([0, 0, 0], dtype=np.float64)
        for idx in range(X_batch.shape[0]):
            img_label = draw_labels(X_batch[idx], img_classed[idx], source.label_colors)
            gt_label = draw_labels(X_batch[idx], gt_batch[idx], source.label_colors)
            # --------------------------------------- #
            # Save sample output
            # cv2.imwrite(output_dir + '{}.jpg'.format(idx+1), img_label)
            # cv2.imwrite(output_dir + 'gt_{}.jpg'.format(idx+1), gt_label)
            # cv2.imwrite(output_dir + 'source_{}.jpg'.format(idx+1), X_batch[idx])

            # --------------------------------------- #
            # Evalute model
            batch_acc += cal_accuracy(img_classed[idx], gt_batch[idx])
        acc += batch_acc
        num_img += X_batch.shape[0]

        if num_img % 120 == 0:
            print('')
            print('Pixel accuracy: {} ---- Mean pixel accuracy: {} ---- Mean IU: {}'.format(acc[0]/num_img, 
                                                                                            acc[1]/num_img, acc[2]/num_img))

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/dataset/FCN/saved_model/20200213-075842/epoch120.ckpt



[i] Processing samples:   8%|▊         | 10/121 [01:42<18:57, 10.24s/batches]


Pixel accuracy: 0.7712972005208335 ---- Mean pixel accuracy: 0.5282190702683793 ---- Mean IU: 0.45237972151678907



[i] Processing samples:  17%|█▋        | 20/121 [03:25<17:14, 10.24s/batches]


Pixel accuracy: 0.7682616357089711 ---- Mean pixel accuracy: 0.5249098607863866 ---- Mean IU: 0.4499942528574321


...


[i] Processing samples:  91%|█████████ | 110/121 [18:44<01:51, 10.14s/batches]


Pixel accuracy: 0.7794932594841911 ---- Mean pixel accuracy: 0.514467507862231 ---- Mean IU: 0.4413584805669059



[i] Processing samples:  99%|█████████▉| 120/121 [20:25<00:10, 10.11s/batches]


Pixel accuracy: 0.7798293063970376 ---- Mean pixel accuracy: 0.5152852078182255 ---- Mean IU: 0.4421485734843674



[i] Processing samples: 100%|██████████| 121/121 [20:32<00:00,  9.37s/batches]


In [26]:
print (acc / source.num_testing)

[0.77951162 0.51481988 0.44164523]
